<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/08_sentiment_analysis_with_LLM(Xwin_LM_13B_V0_1_GPTQ).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLMを使ってみる
* 今日は、とりあえず、LLMを使ってみる。

## 今日の授業の目的
* いまどのくらい手軽にLLMを使えるようになっているかを、とりあえず体感する。
* 技術的な詳細は次回以降学んでいくことにして、とにかく使ってみる。

* **ランタイムのタイプをGPUに設定しておくこと。**

## 例題: LLMによる感情分析(sentiment analysis)
* 今日は、WRIMEというデータセットを使って、LLMに感情分析させてみる。
* 感情分析とは、テキストが表す感情を分析するタスク。
 * ポジティブな感情か、ネガティブな感情かの2値分類タスクとして解くことが多い。
 * 今日は、ニュートラルな感情も含めた3値分類問題として解くことにする。
* LLMとしてはXwin-LM-13B-V0.1を使う。
 * プロンプトを使ったテキスト生成によって感情分析の問題を解く。

## 準備
* Hugging Faceの各種ライブラリを使えば、簡単なコードを書くだけでLLMを使える。

### Transformersライブラリのインストール
* https://huggingface.co/docs/transformers/index

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.9 MB/s eta 0:00:00


### Datasetsライブラリのインストール
* https://huggingface.co/docs/datasets/index

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00


### Accelerateライブラリのインストール
* https://huggingface.co/docs/accelerate/index

In [3]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.8 MB/s eta 0:00:00


### 量子化されたモデルを使うためのライブラリAutoGPTQのインストール
* https://huggingface.co/docs/optimum/llm_quantization/usage_guides/quantization
 * https://huggingface.co/blog/gptq-integration

In [4]:
!pip install auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 8.6 MB/s eta 0:00:00


**ここでランタイムを再起動する。**

### インポート

In [1]:
import os
import torch
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.manual_seed(0)

## データセット
* Ver. 2 の方を使う。
 * WRIME: 主観と客観の感情分析データセット https://github.com/ids-cv/wrime
* Hugging Faceのdatasets hubに登録されているので、簡単に扱うことができる。

### WRIMEデータセットの取得
* training 30,000件、validation 2,500件、test 2,500件。
* 最初だけ、ダウンロードに時間がかかる。

In [2]:
dataset = load_dataset("shunk031/wrime", "ver2")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'user_id', 'datetime', 'writer', 'reader1', 'reader2', 'reader3', 'avg_readers'],
        num_rows: 30000
    })
    validation: Dataset({
        features: ['sentence', 'user_id', 'datetime', 'writer', 'reader1', 'reader2', 'reader3', 'avg_readers'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['sentence', 'user_id', 'datetime', 'writer', 'reader1', 'reader2', 'reader3', 'avg_readers'],
        num_rows: 2500
    })
})

In [4]:
dataset["validation"][0]

{'sentence': '建設中の建物の利用目的も変更になるだろうなあ。',
 'user_id': '26',
 'datetime': '2020/5/10 10:59',
 'writer': {'joy': 0,
  'sadness': 3,
  'anticipation': 1,
  'surprise': 3,
  'anger': 0,
  'fear': 3,
  'disgust': 0,
  'trust': 0,
  'sentiment': 0},
 'reader1': {'joy': 0,
  'sadness': 0,
  'anticipation': 0,
  'surprise': 0,
  'anger': 0,
  'fear': 0,
  'disgust': 2,
  'trust': 0,
  'sentiment': 0},
 'reader2': {'joy': 0,
  'sadness': 1,
  'anticipation': 1,
  'surprise': 0,
  'anger': 0,
  'fear': 0,
  'disgust': 0,
  'trust': 0,
  'sentiment': 0},
 'reader3': {'joy': 0,
  'sadness': 0,
  'anticipation': 0,
  'surprise': 0,
  'anger': 0,
  'fear': 2,
  'disgust': 0,
  'trust': 0,
  'sentiment': -1},
 'avg_readers': {'joy': 0,
  'sadness': 0,
  'anticipation': 0,
  'surprise': 0,
  'anger': 0,
  'fear': 1,
  'disgust': 1,
  'trust': 0,
  'sentiment': 0}}

### 正解ラベルの確認

* 今回は、`avg_readers`の`sentiment`を正解ラベルとして使用する。

In [5]:
from collections import Counter

labels = []
for example in dataset["validation"]:
  labels.append(example["avg_readers"]["sentiment"])
Counter(labels)

Counter({0: 892, -1: 715, 1: 764, -2: 69, 2: 60})

* 元のデータでは５値。
* 今回は、-2と-1をnegativeとして、2と1をpositiveとして、それぞれまとめることにする。
 * これで3値分類の問題になる。
 * このための前処理は後で行う。

## LLM


* 今回は、Xwin-LM-13B-V0.1を使う。
 * https://huggingface.co/Xwin-LM/Xwin-LM-13B-V0.1
* だが、Google Colab無料版では、この元のモデルは大きすぎて使えない・・・。
* そこで、量子化された下記のモデルを代わりに使う。
 * https://huggingface.co/TheBloke/Xwin-LM-13B-V0.1-GPTQ

### Xwin-LM-13B-V0.1-GPTQの取得
* モデルのダウンロードに少し時間がかかる。
* `AutoGPTQForCausalLM`クラスについては、以下を参照。
 * https://github.com/PanQiWei/AutoGPTQ/blob/main/auto_gptq/modeling/auto.py

* safetensorsについては、以下を参照。
 * https://huggingface.co/docs/diffusers/using-diffusers/using_safetensors

* `trust_remote_code`については、[ここ](https://huggingface.co/docs/transformers/model_doc/auto)に以下のような説明がある。

> Whether or not to allow for custom models defined on the Hub in their own modeling files. This option should only be set to True for repositories you trust and in which you have read the code, as it will execute code present on the Hub on your local machine.



In [7]:
from auto_gptq import AutoGPTQForCausalLM

model_name = "TheBloke/Xwin-LM-13B-V0.1-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoGPTQForCausalLM.from_quantized(
    model_name,
    use_safetensors=True,
    inject_fused_attention=False,
    device="cuda:0",
    #trust_remote_code=True,
    )
model.eval()

LlamaGPTQForCausalLM(
  (model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 5120, padding_idx=0)
      (layers): ModuleList(
        (0-39): 40 x LlamaDecoderLayer(
          (self_attn): LlamaAttention(
            (rotary_emb): LlamaRotaryEmbedding()
            (k_proj): GeneralQuantLinear(in_features=5120, out_features=5120, bias=True)
            (o_proj): GeneralQuantLinear(in_features=5120, out_features=5120, bias=True)
            (q_proj): GeneralQuantLinear(in_features=5120, out_features=5120, bias=True)
            (v_proj): GeneralQuantLinear(in_features=5120, out_features=5120, bias=True)
          )
          (mlp): LlamaMLP(
            (act_fn): SiLUActivation()
            (down_proj): GeneralQuantLinear(in_features=13824, out_features=5120, bias=True)
            (gate_proj): GeneralQuantLinear(in_features=5120, out_features=13824, bias=True)
            (up_proj): GeneralQuantLinear(in_features=5120, out_features=13824, bias=Tru

## プロンプト
* LLMがうまく感情分析をしてくれそうなプロンプトを考える。
 * 下はあくまで一つの例。

### プロンプト作成用のヘルパ関数

In [16]:
B_INST, E_INST = "", "答え："
B_SYS, E_SYS = "\n", "\n\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントで、人の感情を察するのが得意です。"

def make_prompt(example):
  sentence = example['sentence']
  text = "「" + sentence + "」と言っている人の気持ちは、どのように説明できますか。"
  text += "また、その気持ちを１単語で言うと、次のどちらですか：「嬉しい」、「悲しい」。"
  text += "どちらでもなければ、「どちらでもない」と答えてください。"
  prompt = "{b_inst} {system}{prompt} {e_inst} ".format(
      b_inst=B_INST,
      system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
      prompt=text,
      e_inst=E_INST,
      )
  example['sentence'] = prompt
  return example

* 元のテキストをプロンプトに一括変換する。

In [17]:
validation_set = dataset["validation"].map(make_prompt)

In [18]:
print(validation_set[0]["sentence"])

 
あなたは誠実で優秀な日本人のアシスタントで、人の感情を察するのが得意です。


「建設中の建物の利用目的も変更になるだろうなあ。」と言っている人の気持ちは、どのように説明できますか。また、その気持ちを１単語で言うと、次のどちらですか：「嬉しい」、「悲しい」。どちらでもなければ、「どちらでもない」と答えてください。 答え： 


* プロンプトをあらかじめトークン化しておく。

In [19]:
validation_set = validation_set.map(lambda samples: tokenizer(samples['sentence']), batched=True)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

* `input_ids`というフィールドにトークン化の結果が格納されている。

In [20]:
print(validation_set[0]["input_ids"])

[1, 259, 13, 30641, 30371, 30366, 30449, 235, 173, 163, 31525, 30499, 232, 135, 173, 31701, 30371, 30325, 30346, 30313, 30199, 30310, 30373, 30255, 30369, 30203, 30279, 30499, 30330, 30313, 30199, 233, 135, 162, 30993, 30396, 232, 178, 162, 30427, 30332, 30199, 30458, 31050, 31474, 30499, 30427, 30267, 13, 13, 13, 30481, 30886, 31770, 30275, 30199, 30886, 30834, 30199, 31107, 30406, 30895, 30210, 30723, 31786, 31100, 30353, 30371, 30332, 30955, 31206, 30465, 30371, 30641, 30267, 30482, 30364, 31243, 30665, 30466, 30298, 30332, 30313, 30199, 31648, 31695, 30644, 30449, 30330, 31250, 30199, 30787, 30465, 30353, 235, 173, 175, 30592, 30499, 30538, 30441, 30427, 30412, 30267, 30441, 30366, 30330, 31110, 30199, 31648, 31695, 30644, 30396, 31936, 232, 144, 155, 30968, 30499, 31243, 30465, 30364, 30330, 30936, 30199, 31250, 30644, 30513, 30499, 30427, 30412, 30383, 30481, 232, 175, 140, 30326, 30298, 30482, 30330, 30481, 233, 133, 181, 30326, 30298, 30482, 30267, 31250, 30644, 30513, 30499, 3

In [21]:
tokenizer.convert_ids_to_tokens([1])

['<s>']

## 感情分析

### 5値を3値に変換するヘルパ関数

In [22]:
def get_sentiment(example):
  sentiment_dic = {-2:'悲しい', -1:'悲しい', 0:'どちらでもない', 1:'嬉しい', 2:'嬉しい'}
  sentiment = example['avg_readers']['sentiment']
  return sentiment_dic[sentiment]

### 感情分析の実行

In [23]:
for i in range(10):
  print(f'[{i+1}]' + '-'*80)
  instance = validation_set[i]
  prompt = instance["sentence"]
  with torch.no_grad():
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    output_ids = model.generate(
        input_ids=token_ids.to(model.device),
        max_new_tokens=256,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
  output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)
  print(f"{prompt}\nprediction:\t{output}")
  print(f"ground truth:\t「{get_sentiment(instance)}」")
  print('-'*80)

[1]--------------------------------------------------------------------------------
 
あなたは誠実で優秀な日本人のアシスタントで、人の感情を察するのが得意です。


「建設中の建物の利用目的も変更になるだろうなあ。」と言っている人の気持ちは、どのように説明できますか。また、その気持ちを１単語で言うと、次のどちらですか：「嬉しい」、「悲しい」。どちらでもなければ、「どちらでもない」と答えてください。 答え： 
prediction:	「悲しい」
ground truth:	「どちらでもない」
--------------------------------------------------------------------------------
[2]--------------------------------------------------------------------------------
 
あなたは誠実で優秀な日本人のアシスタントで、人の感情を察するのが得意です。


「演劇とかも同時並行でやっている。演劇の危機にどう思ってるか知りたかった。」と言っている人の気持ちは、どのように説明できますか。また、その気持ちを１単語で言うと、次のどちらですか：「嬉しい」、「悲しい」。どちらでもなければ、「どちらでもない」と答えてください。 答え： 
prediction:	「悲しい」
ground truth:	「どちらでもない」
--------------------------------------------------------------------------------
[3]--------------------------------------------------------------------------------
 
あなたは誠実で優秀な日本人のアシスタントで、人の感情を察するのが得意です。


「犬夜叉には腐女子っていないのかな？というかあの漫画はＮＬ向けの漫画なのかも。」と言っている人の気持ちは、どのように説明できますか。また、その気持ちを１単語で言うと、次のどちらですか：「嬉しい」、「悲しい」。どちらでもなければ、「どち

# 本日の課題
* もっとうまくLLMに感情分析をさせるプロンプトを考えてみよう。